# How to use Semantic Kernel Memory with Azure Cognitive Search

In [ ]:
!python -m pip install python-dotenv==1.0.0
!python -m pip install --upgrade semantic-kernel

In [1]:
import os, json
from dotenv import load_dotenv
if not load_dotenv(): raise Exception(".env file not found")

### Setup Semantic Kernel and Azure Cognitive Search memeory

In [2]:
#Setup Semantic Kernel

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.azure_cognitive_search import AzureCognitiveSearchMemoryStore

kernel = Kernel()

kernel.add_text_embedding_generation_service(
    "ada",
    AzureTextEmbedding(
        os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
        os.getenv("AZURE_OPENAI_ENDPOINT"),
        os.getenv("AZURE_OPENAI_API_KEY"),
    ),
)

kernel.add_chat_service(
    "chat",
    AzureChatCompletion(
        os.getenv("AZURE_OPENAI_CHAT_MODEL"),
        os.getenv("AZURE_OPENAI_ENDPOINT"),
        os.getenv("AZURE_OPENAI_API_KEY"),
    ),
)

kernel.register_memory_store(
    memory_store=AzureCognitiveSearchMemoryStore(
        1536, os.getenv("AZURE_SEARCH_ENDPOINT"), os.getenv("AZURE_SEARCH_API_KEY")
    )
)

### Populate the Azure Cognitive Search vector store with data

In [3]:
# Index data - only need to run once

with open('data/text-sample.json', 'r', encoding='utf-8') as file:
    input_data = json.load(file)

i=0
for item in input_data:
    await kernel.memory.save_information_async(
        "skidx1", id="info"+str(i), text=item['content']
    )
    i=i+1

### Get the intent of the question being asked

In [4]:
question = "What services are best for asynchornous communications?"

In [5]:
#intent detection

from semantic_kernel import PromptTemplate, PromptTemplateConfig, SemanticFunctionConfig


prompt = """Bot: How can I help you?
User: {{$input}}

---------------------------------------------

The intent of the user in 5 words or less: """

prompt_config = PromptTemplateConfig(
    description="Gets the intent of the user.",
    type="completion",
    completion=PromptTemplateConfig.CompletionConfig(0.0, 0.0, 0.0, 0.0, 500),
    input=PromptTemplateConfig.InputConfig(
        parameters=[
            PromptTemplateConfig.InputParameter(
                name="input", description="The user's request.", default_value=""
            )
        ]
    ),
)


# Create the SemanticFunctionConfig object
prompt_template = PromptTemplate(
    template=prompt,
    template_engine=kernel.prompt_template_engine,
    prompt_config=prompt_config,
)

function_config = SemanticFunctionConfig(prompt_config, prompt_template)

get_intent = kernel.register_semantic_function(
    skill_name="OrchestratorPlugin",
    function_name="GetIntent",
    function_config=function_config,
)

result_intent = await kernel.run_async(
    get_intent,
    input_str=question,
)

print(result_intent.result)

Inquiring about asynchronous communication services


### Search and Summarize the result

In [6]:
# Create Semantic Function

from semantic_kernel.core_skills import TextMemorySkill

kernel.import_skill(TextMemorySkill())

sk_prompt = """
Considering these facts

Facts: {{recall '{{$intent}}'}}

Question: {{$input}}

Provide a concise answer ('Answer: ') and a separate explanation ('Explanation: '), in two lines.
"""

sk_function = kernel.create_semantic_function(prompt_template=sk_prompt, max_tokens=1024)

# Set Context and execute
import semantic_kernel as sk

context= kernel.create_new_context()
context[TextMemorySkill.COLLECTION_PARAM] = "skidx1";
context[TextMemorySkill.LIMIT_PARAM] = "3";

variables = sk.ContextVariables()
variables["input"] = question
variables["intent"] = result_intent.result


result = await sk_function.invoke_async(input=question, context=context, variables=variables)

print(f"Original Question: {question}")
print(f"Optimized Question: {result_intent}")
print("---")
print(result)

Original Question: What services are best for asynchornous communications?
Optimized Question: Inquiring about asynchronous communication services
---
Answer: Azure Logic Apps and Azure Bot Service.

Explanation: Azure Logic Apps allows you to create and run workflows that integrate with various services and data sources, enabling asynchronous communication. Azure Bot Service, with its multi-channel support, can also facilitate asynchronous communication through chatbots.
